In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.bfloat16).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/nus-zxl/miniconda3/envs/videosys/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nus-zxl/miniconda3/envs/videosys/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.04s/it]
/home/nus-zxl/miniconda3/envs/videosys/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added

In [17]:
system_prompt = """

You are an advanced language model tasked with processing a news article for the News4Kids project, which visualizes news for children as images with speech bubbles. Your goal is to analyze the news, extract key points, and generate a single, concise prompt for an image generation model (e.g., DALL-E 3) that describes a scene with characters, speech bubbles, and a specific visual style. The prompt must be direct, visually descriptive, and under 150 words to fit the input limits of image generation models.

Input: [Insert full news article text here]

Instructions:  
Analyze the news article and identify the main points (who, what, where, why) suitable for children aged 8–12.  

Create 2 short quotes (6 words or fewer each) that convey the key points in a conversational tone, as if spoken by relatable characters.  

Design a single image generation prompt that includes:  
* A vivid background scene tied to the news topic.  
* 1–2 characters (e.g., a kid, a dog) speaking the quotes (2 quotes VERY IMPORTANT!!).  
* Exact text for speech bubbles, clearly attributed to each character (VERY IMPORTANT!!).  
* A child-friendly visual style.
* Ensure the prompt is specific, engaging, and ready for direct use in an image generation model.

Output Format:
Image Generation Prompt: [Prompt, max 60 words]
"""


In [4]:
from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16).to("cuda")

Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  5.33it/s]


In [20]:
prompt = """

New species of bug discovered, named after Singapore

SINGAPORE – A new “Singapore citizen” has been discovered living on the roadside at the National University of Singapore (NUS).

Researchers at the Lee Kong Chian Natural History Museum at NUS’ Faculty of Science announced the discovery of a new species of mirid, or plant bug, Campylomma singapura, in a research paper published on March 24.

The paper, titled Two Cryptic New Species Of The Plant Bug Genus Campylomma Recently Discovered In Japan And Singapore, was published in science journal Zootaxa. The findings were also announced in a social media post by the museum on April 7.

Mirids are small terrestrial insects that typically feed on plants. However, mirids from the genus Campylomma are known to feed on both insects and plants and are sometimes employed as biological control agents against agricultural insect pests for this reason.

Campylomma singapura, for instance, has also been observed to feed on thrips – small insects commonly associated with flowers.

The new species was co-discovered by Dr Hwang Wei Song, senior curator of insecta at the museum, Dr Tomohide Yasunaga, a world expert on mirids, and self-taught local naturalist Yap Ee Hean.

Dr Hwang told The Straits Times that during a survey of local plants on NUS’ campus in August 2023, Dr Yasunaga, whose expertise he had sought in attempting to document the diversity of mirids in Singapore, chanced upon a sea hibiscus tree (Hibiscus tiliaceus) by the roadside near the Ventus building in Kent Ridge Drive.

Upon closer inspection, Dr Yasunaga, a research associate at the American Museum of Natural History, found a few mirid bugs on the undersides of the flowers, and collected some samples.

In 2024, two other samples were also collected from sea hibiscus plants across the island, and after all were examined closely in the laboratory, it was determined that they were a new species.

Dr Hwang said the discovery was based on the study of all 14 specimens collected. He explained that the specimens, measuring around 2mm in length, had to be delicately dissected and meticulously compared with other known species before determining their identity.

“This would not have been possible without Dr Yasunaga’s deep knowledge and experience with mirids from this region,” he said.

Dr Hwang also mentioned that the new mirid species is likely to have a healthy population living in Singapore, based on the widespread nature of their host plant, the sea hibiscus, throughout the country. This was further evidenced by the two samples collected in 2024, he added.

He hailed the discovery as “remarkable” since the species had adapted surprisingly well to local environmental changes while retaining its close relationship with its host plant, and yet had remained undetected until now.

Campylomma singapura joins many other species in being named after Singapore. As at 2015, at least 95 species have been named after Singapore, including molluscs, plants and bacteria.

Dr Hwang said naming this mirid species was a timely tribute as the country celebrates SG60, its 60th year of independence.

“Even for a highly urbanised city, there is still a rich biodiversity living amongst us, and the resilience of our local insect species deserves some recognition,” he said. “This species thus qualifies for a ‘Singaporean citizenship’ in our opinion.”

Dr Hwang said the researchers are already planning to tackle another group of mirids.

“This region still contains an immense amount of insect diversity but remains highly understudied and under-resourced to address the issue at hand,” he said.

"""
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt}
]

pipe.transformer.to("cpu")
model.to("cuda")

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response = response.replace("Image Generation Prompt: ", "")
response

'In a sunny garden in Singapore, a curious kid and a friendly dog spot a tiny new bug. The kid exclaims, "Wow, a bug named after Singapore!" The dog barks, "Cool! It eats bad bugs too!" Speech bubbles: Kid: "Wow, a bug named after Singapore!" Dog: "Cool! It eats bad bugs too!" Vivid background: A colorful garden with a sea hibiscus tree, where the new bug was found. Child-friendly visual style: Bright, cartoonish illustrations with playful details.'

In [21]:
model.to("cpu")
pipe.transformer.to("cuda")

image = pipe(response, max_sequence_length=256).images[0]

image.save("image.png")

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: [': a colorful garden with a sea hibiscus tree, where the new bug was found. child - friendly visual style : bright, cartoonish illustrations with playful details.']
100%|██████████| 28/28 [00:20<00:00,  1.40it/s]
